# 逐次意思決定問題における統計的複雑度

参考：
* [The Statistical Complexity of Interactive Decision Making](https://arxiv.org/abs/2112.13487)

教師あり学習ではVC次元やラデマッハ複雑度などの統計的複雑度によって統計的学習問題の難しさを測ることができます．
逐次意思決定問題では同じようなことはできないでしょうか？逐次意思決定問題の場合はそんなに単純ではありません（探索があるためです．実際，VC次元を単純に使ってもうまくいきません．[BANDIT_General_Eluder_dimension.ipynb](BANDIT_General_Eluder_dimension.ipynb)参照）


また，逐次意思決定問題は問題設定も多種多様です．Bandit，RL, POMDPなど，様々な問題設定が考えられ，それぞれに適切な統計的複雑度が存在しそうです．
そこで，今回は多様な逐次意思決定問題を含む，一般的な問題設定と，そこでの統計的複雑度について学びます．

## Decision Making with Structured Observations (DMSO)

DMSOは一般的な逐次意思決定問題のフレームワークとして提案されました．

準備：
* $T$ラウンドの学習を考えます．それぞれのラウンドで，
* 学習者は意思決定$\pi^{(t)}\in \Pi$を選択します．
* 環境は報酬$r^{(t)} \in \mathcal{R} \subseteq \mathbb{R}$と観測$o^{(t)}\in \mathcal{O}$を，意思決定に応じて選択します．報酬と観測は学習者によって観測されます．
* また，$r^{(t)}, o^{(t)}$のペアは未知の分布$M^{\star}\left(\pi^{(t)}\right)$からサンプルされるとします．
ここで，$M^{\star}: \Pi \rightarrow \Delta(\mathcal{R} \times \mathcal{O})$は意思決定を結果にマップする**モデル**です．
* 学習と関数近似を有効利用するために，学習者はモデルのクラス$\mathcal{M}$にアクセスできるとします．また，$M^\star \in \mathcal{M}$を仮定します．$\mathcal{M}$は例えば線形モデルやニューラルネットワーク，ランダムフォレストなど，なんでもいいです．
* $\mathbb{E}^{M, \pi}[\cdot]$ は$(r, o) \sim M(\pi)$の期待値を表すとします．
    * $f^M(\pi) := \mathbb{E}^{M, \pi}[r]$は期待報酬として，$\pi_M:= \arg\max_{\pi \in \Pi} f^M(\pi)$は最大の期待報酬を与える意思決定とします．
* $\Pi_{\mathcal{M}}:=\left\{\pi_M \mid M \in \mathcal{M}\right\}$はそれぞれのモデルに対しての最適意思決定の集合とします．
* $\mathcal{F}_{\mathcal{M}}:=\left\{f^M \mid M \in \mathcal{M}\right\}$はそれぞれのモデルに対する期待報酬関数の集合とします．
* 学習者の性能は次のリグレットで測定するとします：

$$\operatorname{Reg}_{\mathrm{DM}}:=\sum_{t=1}^T \mathbb{E}_{\pi^{(t)} \sim p^{(t)}}\left[f^{\star}\left(\pi^{\star}\right)-f^{\star}\left(\pi^{(t)}\right)\right]$$

ここで，
* $f^{\star}=f^{M^{\star}}$および$\pi^{\star}=\pi_{M^{\star}}$としました．また，$p^{(t)} \in \Delta(\Pi)$は学習者の意思決定の分布です．

このDMSOはいろいろな逐次意思決定問題を内包する一般的な問題設定です．

---

**例：構造的バンディット**

$\pi^{(t)}$をアーム，$\Pi$をアームの空間とすれば，$\mathcal{O}=\{\varnothing\}$のとき，DMSOは構造的バンディット問題と同じです．

さらに次を含みます：
* $\Pi=\{1,\dots, A\}$かつ$\mathcal{F}_{\mathcal{M}}=\mathbb{R}^A$のとき，有限アームバンディットと同じです．
* $\Pi \in \mathbb{R}^d$かつ$\mathcal{F}_{\mathcal{M}}$が線形関数のクラスのとき，線形バンディットと同じです．
* 他にもいろいろ

---

**例：オンライン強化学習**

$M=\left\{\left\{\mathcal{S}_h\right\}_{h=1}^H, \mathcal{A},\left\{P_h^M\right\}_{h=1}^H,\left\{R_h^M\right\}_{h=1}^H, d_1\right\}$を有限MDPとします．

エピソードのスタート時に，確率的な非定常方策$\pi=\left(\pi_1, \ldots, \pi_H\right) \in \Pi_{\mathrm{RNS}}$を選択するとします．そして$|pi$の評価は

$$f^M(\pi):=\mathbb{E}^{M, \pi}\left[\sum_{h=1}^H r_h\right]$$

によって行われるとします．
このとき，例えば$M^\star$をMDP，$\mathcal{M}$をMDPのクラスとすれば，これはモデルベース強化学習を含みます．また，$M^\star$を最適Q関数として，$\mathcal{M}$をQ関数のクラスとすれば，モデルフリー強化学習を含むことになります．


## Decision-Estimation Coefficient (DEC)

上のDSMOフレームワークについて，次のDEC複雑度を導入します．
モデルクラス$\mathcal{M}$とノミナルモデル$\bar{M}$に対して，

$$
\operatorname{dec}_\gamma(\mathcal{M}, \bar{M})=\inf _{p \in \Delta(\Pi)} \sup _{M \in \mathcal{M}} \mathbb{E}_{\pi \sim p}[\underbrace{f^M\left(\pi_M\right)-f^M(\pi)}_{\text {regret of decision }}-\gamma \cdot \underbrace{D_{\mathrm{H}}^2(M(\pi), \bar{M}(\pi))}_{\text {estimation error for obs. }}],
$$

とします．ここで，$\gamma > 0$はスケール用のパラメータ，$D_{\mathrm{H}}^2(\mathbb{P}, \mathbb{Q})=\int(\sqrt{d \mathbb{P}}-\sqrt{d \mathbb{Q}})^2$はHellinger距離です．

また，$\operatorname{dec}_\gamma(\mathcal{M})=\sup _{\bar{M} \in \mathcal{M}} \operatorname{dec}_\gamma(\mathcal{M}, \bar{M})$とします．つまり，

$$
\operatorname{dec}_\gamma(\mathcal{M})= \sup_{\bar{M} \in \mathcal{M}}\inf _{p \in \Delta(\Pi)} \sup _{M \in \mathcal{M}} \mathbb{E}_{\pi \sim p}[\underbrace{f^M\left(\pi_M\right)-f^M(\pi)}_{\text {regret of decision }}-\gamma \cdot \underbrace{D_{\mathrm{H}}^2(M(\pi), \bar{M}(\pi))}_{\text {estimation error for obs. }}],
$$

です．

**DECの説明**

DECは次のMin-maxゲームの値になります．
* ゲームの中身の値を$L(p, M, \bar{M}) = \mathbb{E}_{\pi \sim p}[{f^M\left(\pi_M\right)-f^M(\pi)}-\gamma \cdot D_{\mathrm{H}}^2(M(\pi), \bar{M}(\pi))]$と書くことにします．学習者$(p)$は$L(p, M, \bar{M})$を最小化しようとし，モデル$(M)$は$L(p, M, \bar{M})$を最大化しようとします．
    * ここで，$\bar{M}$は，一般に，「ノミナルモデル（真のモデルっぽいやつ）」を表します．
* ゲームの目標値は「ノミナルモデルについての推定誤差」によってコントロールされます．推定誤差の比重は，Upper boundではDECを最小にする$\gamma$を使い，Lower boundではDECを最大にする$\gamma$を使います．

**直感的な説明：**

逐次意思決定問題のアルゴリズムの目標は$p \in \Delta(\Pi)$の中から適切な分布を見つけることです．
[UCBアルゴリズム](RL_UCB_H_regret_proof.ipynb)などを考えればわかりますが，最適な$p$を見つけるために，一般に，アルゴリズムでは意思決定と，そして方策を更新するためのデータの収集の両方を実現します．
DECゲームを小さくするアルゴリズムを考えると，その正当性がわかります．すなわち，「今までのデータを使ってノミナルモデル$\bar{M}$を作り，DECのMin-maxゲームを解くことで方策$p$を得る」ことを考えてみましょう．
このとき，DECが有界ならば，そのようなアルゴリズムは，1.少ないリグレット もしくは 2. 真のモデルについての何らかの情報 を得ることになります．

なぜなら，
1. $M$は真のモデルを含む最悪ケースなので，真のモデルでのリグレットは一項目よりマシ
2. ２項目より，$M$はなるべくノミナルモデルに近づこうとし，$\pi$はそこから遠ざかろうとするので，ノミナルモデルの欠点を探そうとします．
<!-- 一方で，対象の環境で意思決定をした場合，「意思決定がどれくらい最適な$p$と乖離しているのか？」と「その意思決定によってどれくらい環境についての情報が集まるのか？」の関係は，事前にわかりません． -->

<!-- * １項目：１項目は$\pi$の$M$におけるリグレットです．
* ２項目：２項目は$M(\pi)$と$\bar{M}(\pi)$の区別の付きやすさを表してます．

つまり，$\operatorname{dec}_\gamma(\mathcal{M})$は次の場合に大きくなります：
* 最悪ケースのリグレットが大きい場合：１項目が大きくなる
* 非常に見分けづらいMDPがある場合：２項目が大きくなる -->


## DECと下界

DECを使うと，DMSOにおけるどんなアルゴリズムも絶対に取ってしまうリグレットの下界が求まります．

まず，$\varepsilon \geq 0$とモデル$\bar{M} \in \mathcal{M}$に対して，次の局所モデルクラスを定義します：
* $\mathcal{M}_{\varepsilon}(\bar{M})=\left\{M \in \mathcal{M}: f^{\bar{M}}\left(\pi_{\bar{M}}\right) + \varepsilon\geq f^M\left(\pi_M\right)\right\}$

つまり，報酬が$\bar{M} + \varepsilon$以上にはならないようなモデルの集合です．
ここで，次の略記を導入します：
$$
\operatorname{dec}_\gamma(\mathcal{M})=\sup _{\bar{M} \in \mathcal{M}} \operatorname{dec}_\gamma(\mathcal{M}, \bar{M}) \quad \text { and } \quad \operatorname{dec}_{\gamma, \varepsilon}(\mathcal{M})=\sup _{\bar{M} \in \mathcal{M}} \operatorname{dec}_\gamma\left(\mathcal{M}_{\varepsilon}(\bar{M}), \bar{M}\right)
$$

そして，
$$
V(\mathcal{M})=\sup _{M, M^{\prime} \in \mathcal{M}} \sup _{\pi \in \Pi} \sup _{A \in \mathscr{R} \otimes \mathscr{O}}\left\{\frac{M(A \mid \pi)}{M^{\prime}(A \mid \pi)}\right\} \vee e
$$
を導入します．これはモデルによって出る報酬と観測の最大の密度比です．例えば１つのモデルでしか見れない報酬や観測がある場合，これは無限大になります．

このとき，次が成立します．

---

$\mathcal{F}_{\mathcal{M}} \subseteq(\Pi \rightarrow[0,1])$なるモデルクラス$\mathcal{M}$について，$C(T):=2^{15} \log (2 T \wedge V(\mathcal{M}))$および$\underline{\varepsilon}_\gamma:=C(T)^{-1} \frac{\gamma}{T}$とする．
このとき，どんなアルゴリズムでも，
$$
\boldsymbol{R e g}_{\mathrm{DM}} \geq(6 C(T))^{-1} \cdot \max _{\gamma>\sqrt{C(T) T}} \min \left\{\left(\operatorname{dec}_{\gamma, \underline{\varepsilon}_\gamma}(\mathcal{M})-15 \delta\right) \cdot T, \gamma\right\}
$$

を確率$\delta /2$以上で成立するようなモデル$M$が$\mathcal{M}$に存在する．

---


## E2Dアルゴリズム

上で導入したDMSOフレームワークに対して，次のE2Dアルゴリズムを考えます：

![E2D](figs/E2D.png)

これはリグレットを上から抑えられることが知られています．次の仮定を導入しましょう．

---

次を満たす$c_\ell > 1$が存在する．
任意の$\bar{M}$と$\gamma > 0, \varepsilon > 0$に対して，

$$
\operatorname{dec}_\gamma\left(\mathcal{M}_{\left(c_{\ell} \cdot \varepsilon\right)}(\bar{M}), \bar{M}\right) \leq c_{\ell} \cdot \operatorname{dec}_\gamma\left(\mathcal{M}_{\varepsilon}(\bar{M}), \bar{M}\right)
$$

これは
$$
\operatorname{dec}_\gamma\left(\mathcal{M}_{\varepsilon}(\bar{M}), \bar{M}\right) \propto \varepsilon^\rho \text { for } \rho \leq 1
$$
である場合は成立します．

このとき，次が成立します．

---

$\mathcal{R}\subseteq [0, 1]$かつ上の仮定が成立しているとします．
$C=O\left(c_{\ell}^2 \log _{c_{\ell}}(T)\right)$ 
および
$\bar{\varepsilon}_\gamma:=48\left(\frac{\gamma}{T} \log (|\mathcal{M}| / \delta)+\sup _{\bar{M} \in \operatorname{co}(\mathcal{M})} \operatorname{dec}_\gamma(\mathcal{M}, \bar{M})+\gamma^{-1}\right)$
とします．このとき，確率$1-\delta$以上で，

$$
\boldsymbol{R e g}_{\mathrm{DM}} \leq C \cdot \min _{\gamma>0} \max \left\{\sup _{\bar{M} \in \operatorname{co}(\mathcal{M})} \operatorname{dec}_\gamma\left(\mathcal{M}_{\bar{\varepsilon}_\gamma}(\bar{M}), \bar{M}\right) \cdot T, \gamma \cdot \log (|\mathcal{M}| / \delta)\right\}
$$

が成立します．

---

